# Prepare robots first  
* run panda repeater on controller pc  

```
ssh panda@192.168.0.172
roslaunch panda_ros_repeater joint_velocity_repeater.launch robot_ip:=192.168.0.13 load_gripper:=false

```

* Keep indy connected to conty to bypass conty-connection delay bug

# initialize notebook

In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt

# Initialize constants

In [2]:
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

gtimer = GlobalTimer.instance()
gtimer.reset()

In [3]:
crob = CombinedRobot(connection_list=(False, False))

connection_list
(False, False)


# initialize graph & ui

In [4]:
if "cam" not in locals():
    cam = None # StereoCamera.instance()

# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, XYZ_RPY_ROBOTS_DEFAULT, crob.custom_limits, start_rviz=True)
ghnd = GeometryHandle(urdf_content)


graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                        urdf_content=urdf_content, combined_robot=crob)
graph.set_camera(cam)
graph.set_cam_robot_collision(_add_cam_poles=False)
graph.set_rviz()

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(graph)
ui_broker.start_server()

# set rviz
graph.set_rviz(crob.home_pose)
ui_broker.set_tables()

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [5]:
from pkg.planner.moveit import moveit_py
from pkg.planner.moveit.moveit_planner import MoveitPlanner

### Set Planner

In [6]:
mplan = MoveitPlanner(
    joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content, 
    robot_names=crob.robot_names, ghnd=ghnd)

In [17]:
mtem_list=[]

### define tool

In [7]:
GROUP_NAME, TOOL_LINK, BASE_LINK  = "indy0", "indy0_tcp", "base_link"
TOOL_XYZRPY = ((0,0,0.14),(np.pi,0,0))
# TOOL_XYZRPY = ((0,0,0),(0,0,0))
# ORIGIN_XYZRPY = ((0,0,0.4), (0,0,0))
ORIGIN_XYZRPY = (( -0.637836,   -0.102947,  0.527019  ), (3.14151544, -0.01124507,  1.55726101))

TOOL_TF = T_xyzrpy(TOOL_XYZRPY)
TOOL_XYZQUAT = T2xyzquat(TOOL_TF)
ORIGIN_TF = T_xyzrpy(ORIGIN_XYZRPY)

# Plane Test

### Show constraints

In [18]:
for mtem in mtem_list:
    graph.remove_geometry(mtem)
mtem_list=[]

generate table - Geometry
generate table - Handle
generate table - Bindergenerate table - Object



In [19]:
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.BOX, name="plane", link_name="base_link",
                                      center=ORIGIN_XYZRPY[0], rpy=ORIGIN_XYZRPY[1], 
                                      dims=(3,3,0.001), color=(1, 1, 0, 0.5),
                                      collision=False))
for mtem in mtem_list:
    graph.add_marker(mtem)

### move to starting pose

In [20]:
Q_init = crob.home_pose
goal_pose = T2xyzquat(np.matmul(ORIGIN_TF, TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=goal_pose[0]+goal_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_ready = trajectory[-1]

In [21]:
graph.show_motion(trajectory)
graph.show_pose(Q_ready)
print("goal pose: {}".format(goal_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

goal pose: ([-0.637803852558136, -0.10137300938367844, 0.3870278596878052], [0.003921231635065741, -0.004029640151390661, 0.7022939191396269, 0.7118649008640737])
Q_ready: [-0.   -2.82  2.18 -0.    0.64  3.14  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


### set manifolds

In [24]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
ctem_list = make_constraint_list(mtem_list)
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list, fix_surface=True, fix_normal=True, radius=1e-5, tol=1e-3)

### search plan

In [28]:
tool_tf = np.matmul(get_tf(TOOL_LINK, list2dict(Q_ready, mplan.joint_names), mplan.urdf_content), T_xyzrpy(TOOL_XYZRPY))
goal_pose = T2xyzquat(np.matmul( \
                                np.matmul(ORIGIN_TF, SE3(np.identity(3), (0.0,0.3,0.0))),
                                TOOL_TF))

In [29]:
trajectory, success = mplan.planner.plan_constrained_py(GROUP_NAME,TOOL_LINK, goal_pose[0]+goal_pose[1], BASE_LINK, Q_ready, timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (40)


In [30]:
graph.show_motion(trajectory, period=0.01)

# Sphere Test

### Show constraints

In [32]:
for mtem in mtem_list:
    graph.remove_geometry(mtem)
mtem_list=[]

In [33]:
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.SPHERE, name="cylinder", link_name="base_link",
                                      center=np.add(ORIGIN_XYZRPY[0], (0.05,0,0)), rpy=ORIGIN_XYZRPY[1], 
                                      dims=(0.1,0.1,0.1), color=(1, 1, 0, 0.5),
                                      collision=False))
for mtem in mtem_list:
    graph.add_marker(mtem)

### check starting pose

In [54]:
Q_init = crob.home_pose
start_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi/2,0,0)),[0]*3)), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=start_pose[0]+start_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_ready = trajectory[-1]

In [55]:
graph.show_motion(trajectory)
graph.show_pose(Q_ready)
print("goal pose: {}".format(goal_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

goal pose: ([-0.5878841280937195, -0.10575991123914719, 0.7170108556747437], [0.7118649008640737, 0.7022939191396269, 0.004029640151390661, -0.003921231635065741])
Q_ready: [-1.07 -0.68 -0.75  1.82 -2.62  0.28  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


### check goal pose

In [56]:
Q_init = crob.home_pose
goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi,0,0)),[0,0.05,-0.05])), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=goal_pose[0]+goal_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)

In [57]:
graph.show_motion(trajectory)
graph.show_pose(Q_ready)
print("goal pose: {}".format(goal_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

goal pose: ([-0.5878841280937195, -0.10575991123914719, 0.7170108556747437], [0.7118649008640737, 0.7022939191396269, 0.004029640151390661, -0.003921231635065741])
Q_ready: [-1.07 -0.68 -0.75  1.82 -2.62  0.28  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


### set manifolds

In [62]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
ctem_list = make_constraint_list(mtem_list)
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list, fix_surface=True, fix_normal=True, radius=1e-5, tol=2e-3)

### search plan

In [63]:
trajectory, success = mplan.planner.plan_constrained_py(GROUP_NAME,TOOL_LINK, goal_pose[0]+goal_pose[1], BASE_LINK, Q_ready, timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (100)


In [64]:
graph.show_motion(trajectory, period=0.01)

# Cylinder Test

### Show constraints

In [65]:
for mtem in mtem_list:
    graph.remove_geometry(mtem)
mtem_list=[]

In [66]:
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.CYLINDER, name="cylinder", link_name="base_link",
                                      center=np.add(ORIGIN_XYZRPY[0], (0.05,0,0)), rpy=ORIGIN_XYZRPY[1], 
                                      dims=(0.1,0.1,0.1), color=(1, 1, 0, 0.5),
                                      collision=False))
for mtem in mtem_list:
    graph.add_marker(mtem)

### check starting pose

In [82]:
Q_init = crob.home_pose
start_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi/2,0,0)),[0]*3)), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=start_pose[0]+start_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_ready = trajectory[-1]

In [83]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])
print("goal pose: {}".format(goal_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

goal pose: ([-0.5852690935134888, 0.08634680509567261, 0.529159426689148], [0.7110548438724447, -0.004839702516332617, -0.00473129427046989, 0.7031039760516312])
Q_ready: [-1.07 -0.68 -0.75  1.82 -2.62  0.28  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


### check goal pose

In [84]:
Q_init = crob.home_pose
goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi/2,0,np.pi/2)),[0.05,0.05,0])), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=goal_pose[0]+goal_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_ready, plannerconfig="RRTConnectkConfigDefault", timeout=1)

In [85]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])
print("goal pose: {}".format(goal_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

goal pose: ([-0.5852690935134888, 0.08634680509567261, 0.529159426689148], [0.7110548438724447, -0.004839702516332617, -0.00473129427046989, 0.7031039760516312])
Q_ready: [-1.07 -0.68 -0.75  1.82 -2.62  0.28  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


### set manifolds

In [86]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
ctem_list = make_constraint_list(mtem_list)
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list, fix_surface=True, fix_normal=True, radius=1e-5, tol=2e-3)

### search plan

In [87]:
trajectory, success = mplan.planner.plan_constrained_py(GROUP_NAME,TOOL_LINK, goal_pose[0]+goal_pose[1], BASE_LINK, Q_ready, timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (92)


In [88]:
graph.show_motion(trajectory, period=0.01)